## 7.查询数据
- 测试文件: mysql -u root -p < init-test-data.sql
- SELECT 1;来测试数据库连接
- 条件查询:
    - 与：SELECT * FROM students WHERE score >= 80 AND gender = 'M';
    - 或：SELECT * FROM students WHERE score >= 90 OR gender = 'M';
    - 非：SELECT * FROM students WHERE NOT class_id = 2;
    - 如果不加括号，条件运算按照NOT、AND、OR的优先级进行，即NOT优先级最高，其次是AND，最后是OR。加上括号可以改变优先级
    - 使用LIKE判断相似:%表示任意字符，例如'ab%'将匹配'ab'，'abc'，'abcd'
- 投影查询:
    - SELECT id, score, name FROM students;
    - 可以给列起个别名: SELECT id, score points, name FROM students WHERE gender = 'M';
- 排序
    - 加上DESC表示“倒序”: SELECT id, name, gender, score FROM students ORDER BY score DESC;
    - 要进一步排序，可以继续添加列名: SELECT id, name, gender, score FROM students ORDER BY score DESC, gender;
    - 如果有WHERE子句，那么ORDER BY子句要放到WHERE子句后面
```
SELECT id, name, gender, score
FROM students
WHERE class_id = 1
ORDER BY score DESC;
```
- 分页查询
```
SELECT id, name, gender, score
FROM students
ORDER BY score DESC
LIMIT 3 OFFSET 0;
使用LIMIT <M> OFFSET <N>分页时，随着N越来越大，查询效率也会越来越低
```
>LIMIT总是设定为pageSize；OFFSET计算公式为pageSize * (pageIndex - 1)
- 聚合查询
    - SELECT COUNT(*) FROM students;查询的结果仍然是一个二维表，只是这个二维表只有一行一列，并且列名是COUNT(*)
    - 设计列名别名为num: SELECT COUNT(*) num FROM students;
    - WHERE: SELECT COUNT(*) boys FROM students WHERE gender = 'M';
    - SUM, AVG, MAX, MIN
    - SELECT AVG(score) average FROM students WHERE gender = 'M';
```
保留小数: SELECT COUNT(*) / 3 FROM students;
取下整: SELECT FLOOR(COUNT(*) / 3) FROM students;
取上整: SELECT CEILING(COUNT(*) / 3) FROM students;
```
    - 分组聚合
        - SELECT COUNT(*) num FROM students GROUP BY class_id;
        - SELECT class_id, COUNT(*) num FROM students GROUP BY class_id;
        - SELECT class_id, gender, COUNT(*) num FROM students GROUP BY class_id, gender;
        - SELECT class_id, AVG(score) class_avg_score FROM students GROUP BY class_id;
        - SELECT class_id, gender, AVG(score) class_avg_score FROM students GROUP BY class_id, gender;
- 多表查询
```
SELECT
    students.id sid,
    students.name,
    students.gender,
    students.score,
    classes.id cid,
    classes.name cname
FROM students, classes;

SELECT
    s.id sid,
    s.name,
    s.gender,
    s.score,
    c.id cid,
    c.name cname
FROM students s, classes c
WHERE s.gender = 'M' AND c.id = 1;
```
- 连接查询
    - INNER JOIN
    
        ```
        SELECT s.id, s.name, s.class_id, c.name class_name, s.gender, s.score
        FROM students s
        INNER JOIN classes c
        ON s.class_id = c.id;
        注意INNER JOIN查询的写法是：
        1. 先确定主表，仍然使用FROM <表1>的语法；
        2. 再确定需要连接的表，使用INNER JOIN <表2>的语法；
        3. 然后确定连接条件，使用ON <条件...>，这里的条件是s.class_id = c.id，表示students表的class_id列与classes表的id列相同的行需要连接；
        4. 可选：加上WHERE子句、ORDER BY等子句。
        ```
    
    - OUTER JOIN
    
    ```
    RIGHT OUTER JOIN返回右表都存在的行。如果某一行仅在右表存在，那么结果集就会以NULL填充剩下的字段
    SELECT s.id, s.name, s.class_id, c.name class_name, s.gender, s.score
    FROM students s
    RIGHT OUTER JOIN classes c
    ON s.class_id = c.id;
    
    LEFT OUTER JOIN则返回左表都存在的行。如果我们给students表增加一行，并添加class_id=5，由于classes表并不存在id=5的行，所以，LEFT OUTER JOIN的结果会增加一行，对应的class_name是NULL：
    先增加一列class_id=5:
    INSERT INTO students (class_id, name, gender, score) values (5, '新生', 'M', 88);
    SELECT s.id, s.name, s.class_id, c.name class_name, s.gender, s.score
    FROM students s
    LEFT OUTER JOIN classes c
    ON s.class_id = c.id;
    
    FULL OUTER JOIN，它会把两张表的所有记录全部选择出来，并且，自动把对方不存在的列填充为NULL：
    SELECT s.id, s.name, s.class_id, c.name class_name, s.gender, s.score
    FROM students s
    FULL OUTER JOIN classes c
    ON s.class_id = c.id;
    ```